In [41]:
def get_image_names_and_delete_mask_and_mirror(All_Folders,num,KK):

     files = filter(os.path.isfile, os.listdir(KK));
     files = [os.path.join(KK, f) for f in files if f] # add path to each file
     files.sort(key=lambda x: os.path.getmtime(x))

     nwefiles = [os.path.splitext(os.path.basename(f)) for f in  files] # add path to each file
     Prefix =  nwefiles[0]
     Prefix = Prefix[0] 
     nwefiles = [(f[0]) for f in nwefiles if f[0].startswith('Mask')]
     nwefiles = [os.path.join(KK, f+'.jpg') for f in  nwefiles] # add path to each file
     [os.remove(f) for f in  nwefiles]
     nwefiles = [os.path.splitext(os.path.basename(f)) for f in  files] # add path to each file
     nwefiles = [(f[0]) for f in nwefiles if f[0].startswith('R_mirror')]
     nwefiles = [os.path.join(KK, f+'.jpg') for f in  nwefiles]
     [os.remove(f) for f in  nwefiles]
     nwefiles = [os.path.splitext(os.path.basename(f)) for f in  files] # add path to each file
     nwefiles = [(f[0]) for f in nwefiles if not f[0].startswith('Mask') and not f[0].startswith('R_mirror')]
     nwefiles = [(int(f[-3:])) for f in nwefiles]
     nwefiles = sorted(nwefiles)
     nwefiles = ["%03d" % x for x in nwefiles]
     files = [os.path.join(KK, Prefix[0:-3] + str(f) + '.jpg') for f in nwefiles]  # add path to each file

     return files


def make_movies_out_of_images(images, video_name, mainfolder, worksheet,xL,yL,wL,hL,xC,yC,wC,hC,xR,yR,wR,hR):
    frame = cv2.imread(os.path.join(mainfolder, images[0]))
    height, width, layers = frame.shape
    video = cv2.VideoWriter(video_name, 0, 40, (width, height))
    fps = 40

    for idx, image in enumerate(images):
        # worksheet.write_row( idx+1,col, mat)
        frame = cv2.imread(os.path.join(Mainfolder, images[idx]))
        video.write(frame)

        cropped_l = frame[yL:yL + hL, xL:xL + wL]  # both opencv and numpy are "row-major", so y goes first
        cropped_r = frame[yR:yR + hR, xR:xR + wR]  # both opencv and numpy are "row-major", so y goes first
        cropped_c = frame[yC:yC + hC, xC:xC + wC]  # both opencv and numpy are "row-major", so y goes first
        worksheet.write_row(idx + 1, col, [np.mean(cropped_l)])
        worksheet.write_row(idx + 501, col, [np.mean(cropped_c)])
        worksheet.write_row(idx + 1001, col, [np.mean(cropped_r)])
    video.release()
    cv2.destroyAllWindows()
    return worksheet
    
    


def makemovieandLightfile(Mainfolder):
    import xlsxwriter  # this is to write xls files 
    import os # this is to get folder information and creat files 
    import cv2 # this is important and is reading and writing images and video 
    import shutil # to remove folder and subfolders

    my_file = os.path.join(Mainfolder+'\Lighttime.xlsx');
    workbook = xlsxwriter.Workbook(my_file)
    worksheet = workbook.add_worksheet()

    col = 0
    All_Folders=[ name for name in os.listdir(Mainfolder) if os.path.isdir(os.path.join(Mainfolder, name)) ]

    for num in range(1,len(All_Folders)): 
     KK = os.path.join(Mainfolder+os.sep, All_Folders[num])
     KKavi =  os.path.join(Mainfolder+os.sep, All_Folders[num]+'.avi')
     #if os.path.isfile(KKavi):
      # continue
     if not os.path.isdir(KK):
       continue
     dirContents = os.listdir(KK)
     if len(dirContents) == 0:
      shutil.rmtree(KK, ignore_errors=False, onerror=None)
      continue
     os.chdir(KK)
     print(num)
     images=get_image_names_and_delete_mask_and_mirror(All_Folders,num,KK)
     if num < -1:
      frame = cv2.imread(os.path.join(Mainfolder, images[0]))
      (xL,yL,wL,hL) = cv2.selectROI('image', frame)
      (xC,yC,wC,hC) = cv2.selectROI('image', frame)
      (xR,yR,wR,hR) = cv2.selectROI('image', frame)
     video_name = (KK + '.avi');
     video_name2 =(KK+'video'+ '.mp4');
     worksheet=make_movies_out_of_images(images, video_name, Mainfolder, worksheet,xL,yL,wL,hL,xC,yC,wC,hC,xR,yR,wR,hR)
     convert_video(video_name, video_name2)
    workbook.close()
   

def make_movies_out_of_images(images, video_name, mainfolder, worksheet,xL,yL,wL,hL,xC,yC,wC,hC,xR,yR,wR,hR):
    frame = cv2.imread(os.path.join(mainfolder, images[0]))
    height, width, layers = frame.shape
    video = cv2.VideoWriter(video_name, 0, 40, (width, height))
    fps = 40

    for idx, image in enumerate(images):
        # worksheet.write_row( idx+1,col, mat)
        frame = cv2.imread(os.path.join(Mainfolder, images[idx]))
        video.write(frame)

        cropped_l = frame[yL:yL + hL, xL:xL + wL]  # both opencv and numpy are "row-major", so y goes first
        cropped_r = frame[yR:yR + hR, xR:xR + wR]  # both opencv and numpy are "row-major", so y goes first
        cropped_c = frame[yC:yC + hC, xC:xC + wC]  # both opencv and numpy are "row-major", so y goes first
        worksheet.write_row(idx + 1, col, [np.mean(cropped_l)])
        worksheet.write_row(idx + 501, col, [np.mean(cropped_c)])
        worksheet.write_row(idx + 1001, col, [np.mean(cropped_r)])
    video.release()
    cv2.destroyAllWindows()
    return worksheet
    
    


def makemovieandLightfile(Mainfolder):
    import xlsxwriter  # this is to write xls files 
    import os # this is to get folder information and creat files 
    import cv2 # this is important and is reading and writing images and video 
    import shutil # to remove folder and subfolders

    my_file = os.path.join(Mainfolder+'\Lighttime.xlsx');
    workbook = xlsxwriter.Workbook(my_file)
    worksheet = workbook.add_worksheet()

    col = 0
    All_Folders=[ name for name in os.listdir(Mainfolder) if os.path.isdir(os.path.join(Mainfolder, name)) ]

    for num in range(1,len(All_Folders)): 
     KK = os.path.join(Mainfolder+os.sep, All_Folders[num])
     KKavi =  os.path.join(Mainfolder+os.sep, All_Folders[num]+'.avi')
     #if os.path.isfile(KKavi):
      # continue
     if not os.path.isdir(KK):
       continue
     dirContents = os.listdir(KK)
     if len(dirContents) == 0:
      shutil.rmtree(KK, ignore_errors=False, onerror=None)
      continue
     os.chdir(KK)
     print(num)
     images=get_image_names_and_delete_mask_and_mirror(All_Folders,num,KK)
     if num < -1:
      frame = cv2.imread(os.path.join(Mainfolder, images[0]))
      (xL,yL,wL,hL) = cv2.selectROI('image', frame)
      (xC,yC,wC,hC) = cv2.selectROI('image', frame)
      (xR,yR,wR,hR) = cv2.selectROI('image', frame)
     video_name = (KK + '.avi');
     video_name2 =(KK+'video'+ '.mp4');
     worksheet=make_movies_out_of_images(images, video_name, Mainfolder, worksheet,xL,yL,wL,hL,xC,yC,wC,hC,xR,yR,wR,hR)
     convert_video(video_name, video_name2)
    workbook.close()
   


